In [1]:
# requirements
import os
import re
import time
import random
import datetime
import schedule
import requests
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from pytz import timezone
from bs4 import BeautifulSoup
from datetime import timedelta,datetime


In [1]:
class Crawler:   
    press_list = {'한국경제':'015', '매일경제':'009', '조선일보':'023', '동아일보':'020', '한겨레신문':'028', '경향신문':'032'}
    press_names = list(press_list.keys())
    press_codes = list(press_list.values())

    formatting = '%Y-%m-%d'
    condition = "\[.*\]|\s-\s.*" 

    now = datetime.now(timezone('Asia/Seoul'))
    str_now = datetime.strftime(now,formatting)
    now_yymmdd = str_now.split('-')
    now_yymmdd = [int (i) for i in now_yymmdd]

    yesterday = now - timedelta(1)
    str_yesterday = datetime.strftime(yesterday,formatting)
    yesterday_yymmdd = str_yesterday.split('-')
    yesterday_yymmdd = [int(i) for i in yesterday_yymmdd] 
    
    def __init__(self):
        pass
    

    def crawling_yesterday(self):
        # 크롤링 할 대상 입력
        for press in self.press_names:
            year, month, day = map(int, self.yesterday_yymmdd[0:3])
            to_crawl_press_this_time = press
            presscode = self.press_list[to_crawl_press_this_time]

            # 6개 신문사 불러오기 
            df = pd.DataFrame(columns=["press","title","date"])
            yesterday_data = []
            if int(month) < 10:
                month = f"0{month}"
            print(f'{to_crawl_press_this_time}의 {month}월 {day}일 기사를 크롤링합니다.')
            print("**************************************")

            if day in tqdm(range(1,32)):
                # URL에 접속 시도, 1 ~ 9일에 대한 URL 접근
                if day < 10:
                    URL = f'https://media.naver.com/press/{presscode}/newspaper?date={year}{month}0{day}'
                    print(URL)
                    response = requests.get(URL)
                    print(response)

                # URL에 접속 시도,10일부터에 대한 URL 접근
                else:
                    URL = f'https://media.naver.com/press/{presscode}/newspaper?date={year}{month}{day}'
                    print(URL)
                    response = requests.get(URL)
                    print(response)

                time.sleep(1)

                # html 파싱 시작
                soup = BeautifulSoup(response.text)
                page = soup.select('div.newspaper_inner')

                # 혹시나 1면도 없으면 신문 없는 날
                if len(page) < 1:
                    print(f'{day}일자는 기사가 없습니다 데헷')
                    print('-' * 25)
                    continue

                # 신문 지면 발견하면 크롤링
                else:
                    print(f'{day}일자 기사를 크롤링합니다.')
                    print('-' * 25)

                    # 하루치 기사를 담을 리스트
                    oneday_titles = []

                    # 지면별 헤드라인 탐색
                    for _ in range(len(page)):
                        page_titles = []
                        title = soup.select('div.newspaper_inner')[_]
                        titles = title.find_all('strong')
                        urls=title.find_all('a')
                        href_list=[]
                        for a in urls:
                            href = a.attrs['href']
                            href_list.append(href)

                        # 헤드라인들 page_title 리스트에 담음
                        for ttl in range(len(titles)):
                            news_title = titles[ttl].text
                            url = href_list[ttl]

                            
                            if len(news_title) < 8:
                                news_title == np.nan
                            if day < 10:
                                news = {'press':to_crawl_press_this_time,'title':news_title, 'date':f'{year}{month}0{day}','url':url}
                            else:
                                news = {'press':to_crawl_press_this_time,'title':news_title, 'date':f'{year}{month}{day}','url':url}

                            yesterday_data.append(news)

            df = df.append(yesterday_data)
            df = df.dropna(axis = 0)
            df.to_csv(f'data/common/{to_crawl_press_this_time}_{year}_{month}_{day}.csv', encoding="UTF-8-sig")
        return print("어제자 일반뉴스 크롤링 및 저장 완료")

    
    def crawling_today(self):
        # 크롤링 할 대상 입력
        for press in self.press_names:
            year, month, day = map(int, self.now_yymmdd[0:3])
            to_crawl_press_this_time = press
            presscode = self.press_list[to_crawl_press_this_time]

            # 6개 신문사 불러오기 
            df = pd.DataFrame(columns=["press","title","date"])
            today_data = []

            if month < 10:
                month = f"0{month}"
            print(f'{to_crawl_press_this_time}의 {month}월 {day}일 기사를 크롤링합니다.')
            print("**************************************")

            if day in tqdm(range(1,32)):
                # URL에 접속 시도,1 ~ 9일에 대한 URL 접근
                if day < 10:
                    URL = f'https://media.naver.com/press/{presscode}/newspaper?date={year}{month}0{day}'
                    print(URL)
                    response = requests.get(URL)
                    print(response)
                # 10일부터에 대한 URL 접근
                else:
                    URL = f'https://media.naver.com/press/{presscode}/newspaper?date={year}{month}{day}'
                    print(URL)
                    response = requests.get(URL)
                    print(response)
                    
                time.sleep(1)
                
                # html 파싱 시작
                soup = BeautifulSoup(response.text)
                page = soup.select('div.newspaper_inner')

                # 혹시나 1면도 없으면 신문 없는 날
                if len(page) < 1:
                    print(f'{day}일자는 기사가 없습니다 데헷')
                    print('-' * 25)
                    continue

                # 신문 지면 발견하면 크롤링
                else:
                    print(f'{day}일자 기사를 크롤링합니다.')
                    print('-' * 25)

                    # 하루치 기사를 담을 리스트
                    oneday_titles = []

                    # 지면별 헤드라인 탐색
                    for _ in range(len(page)):
                        page_titles = []
                        title = soup.select('div.newspaper_inner')[_]
                        titles = title.find_all('strong')
                        urls=title.find_all('a')
                        href_list=[]
                        for a in urls:
                            href = a.attrs['href']
                            href_list.append(href)

                        # 헤드라인들 page_title 리스트에 담음
                        for ttl in range(len(titles)):
                            news_title = titles[ttl].text
                            url = href_list[ttl]
                            if len(news_title) < 8:
                                news_title == np.nan
                            if day < 10:
                                news = {'press':to_crawl_press_this_time,'title':news_title, 'date':f'{year}{month}0{day}','url':url}
                            else:
                                news = {'press':to_crawl_press_this_time,'title':news_title, 'date':f'{year}{month}{day}','url':url}

                            today_data.append(news)

            df = df.append(today_data)
            df = df.dropna(axis = 0)
            if day < 10:
                df.to_csv(f'data/common/{to_crawl_press_this_time}_{year}_{month}_0{day}.csv', encoding="UTF-8-sig")
            else:
                df.to_csv(f'data/common/{to_crawl_press_this_time}_{year}_{month}_{day}.csv', encoding="UTF-8-sig")
        
        return print("오늘자 일반뉴스 크롤링 및 저장 완료.")

    
    def crawling_yesterday_finance(self):
        # 크롤링 할 대상 입력
        yesterday_date = self.str_yesterday
        
        df = pd.DataFrame(columns=["press","title","date"])
        yesterday_data = []  #데이터 담을 빈 리스트
        print(f'네이버금융뉴스의 {yesterday_date}자 기사를 크롤링합니다.')
        print("**************************************")

        url = f'https://finance.naver.com/news/mainnews.naver?date={yesterday_date}'
        print(url)
        response = requests.get(url)
        
        time.sleep(1)

        # html 파싱 시작
        soup = BeautifulSoup(response.text)
        titles = soup.find_all('li', class_='block1')
        url_plus='https://finance.naver.com/'

        if titles:
            for i in titles:
                temp=i.find_all('a')
                #썸네일있는 기사와 없는기사 구분해서 크롤링
                if len(temp)==2:
                    title=temp[1].text
                    url=temp[1].attrs['href']
                    text = re.sub(self.condition, '',title)
                    if len(text) < 8:
                        text == np.nan
                    news = {'press':'네이버금융','title':text, 'date':yesterday_date,'url':url_plus+url}
                    yesterday_data.append(news)
                else:
                    title=temp[0].text
                    url=temp[0].attrs['href']
                    text = re.sub(self.condition, '',title)
                    if len(text) < 8:
                        text == np.nan
                    news = {'press':'네이버금융','title':text, 'date':yesterday_date,'url':url_plus+url}
                    yesterday_data.append(news)
        else:
            print('기사가 없습니다.')

        df = df.append(yesterday_data)
        df = df.dropna(axis = 0)
        df.to_csv(f'data/finance/finance_{yesterday_date}.csv', encoding="UTF-8-sig")
        return print("어제 자 금융뉴스 크롤링 및 저장 완료")

    
    def crawling_today_finance(self):
        # 크롤링 할 대상 입력
        today_date = self.str_now
        
        df = pd.DataFrame(columns=["press","title","date"])
        today_data = []  #데이터 담을 빈 리스트
        print(f'네이버금융뉴스의 {today_date}날짜 기사를 크롤링합니다.')
        print("**************************************")


        url = f'https://finance.naver.com/news/mainnews.naver?date={today_date}'
        print(url)
        response = requests.get(url)
        
        time.sleep(1)

        # html 파싱 시작
        soup = BeautifulSoup(response.text)
        titles = soup.find_all('li', class_='block1')
        url_plus='https://finance.naver.com/'

        if titles:
            for i in titles:
                temp=i.find_all('a')
                #썸네일있는 기사와 없는기사 구분해서 크롤링
                if len(temp)==2:
                    title=temp[1].text
                    url=temp[1].attrs['href']
                    text = re.sub(self.condition, '',title)
                    if len(text) < 8:
                        text == np.nan
                    news = {'press':'네이버금융','title':text, 'date':today_date,'url':url_plus+url}
                    today_data.append(news)
                else:
                    title=temp[0].text
                    url=temp[0].attrs['href']
                    text = re.sub(self.condition, '',title)
                    if len(text) < 8:
                        text == np.nan
                    news = {'press':'네이버금융','title':text, 'date':today_date,'url':url_plus+url}
                    today_data.append(news)
        else:
            print('기사가 없습니다.')

        df = df.append(today_data)
        df = df.dropna(axis = 0)
        df.to_csv(f'data/finance/finance_{today_date}.csv', encoding="UTF-8-sig")
        
        return print("오늘 자 금융뉴스 크롤링 및 저장 완료")
    
            
    def merge(self):
        
        df_all = pd.DataFrame()
        path_of_this_file = os.getcwd()
        
        common_folder = os.listdir(f"{path_of_this_file}/data/common")# 파일들이 있는 폴더명으로 폴더내 파일 목록 확인
        common_folder.remove('daily')

        
        # 파일 병합 및 저장 
        for file in common_folder:
            file = f'{path_of_this_file}/data/common/{file}'
            df = pd.read_csv(file, encoding='utf-8-sig', index_col=0)
            df_all = pd.concat([df_all, df])
               
        df_all.dropna()
        df_all.to_csv(f'{path_of_this_file}/data/common/daily/{self.str_now}.csv', encoding="utf-8-sig")   
        
        # 기존파일 삭제
        for file in common_folder:
            file = f'{path_of_this_file}/data/common/{file}'
            os.remove(file)
        
        return print("어제와 오늘 일반뉴스 파일 병합 및 저장 완료.")

    
    def merge_finance(self):
        
        df_all = pd.DataFrame()
        path_of_this_file = os.getcwd()
        
        finance_folder = os.listdir(f"{path_of_this_file}/data/finance")# 파일들이 있는 폴더명으로 폴더내 파일 목록 확인
        finance_folder.remove('daily')
        
        # 파일 병합 및 저장
        for file in finance_folder:
            file = f'{path_of_this_file}/data/finance/{file}'
            df = pd.read_csv(file, encoding='utf-8-sig', index_col=0)
            df_all = pd.concat([df_all, df])
        
        df_all.to_csv(f'{path_of_this_file}/data/finance/daily/{self.str_now}.csv', encoding="utf-8-sig")   
        
        # 기존파일 삭제
        for file in finance_folder:
            file = f'{path_of_this_file}/data/finance/{file}'
            os.remove(file)
        
        return print("어제와 오늘 금융뉴스 파일 병합 및 저장 완료.")
    


NameError: name 'datetime' is not defined